## **Coleta de dados do Datalake da Basedosdados**
***
##### **Objetivo**: coletar dados sobre série histórica da inflação e venda de combustíveis dos distribuidores, armazenados no data lake público da base dos dados
##### **Resultado**: adição de duas tabelas, uma com série histórica da inflação e outra com venda de combustíveis dos distribuidores no banco de dados 'dados.bd'
***

In [12]:
# Bibliotecas utilizadas
import pandas  as pd
import sqlite3 as sql3

from basedosdados import read_sql
from glob         import glob

In [13]:
# Criando função para ler os arquivos '.sql'
def ler_query( nome_query ):
    
    # Abrindo arquivo de interesse
    arq = open( nome_query, 'r')
    
    # Lendo conteúdo do arquivo e amazenando na variável
    query = arq.read()
    
    # Fechando arquivo
    arq.close()

    # Retornando o conteúdo do arquivo
    return query

In [14]:
# Coletando e armazenando em uma lista os nomes de arquivos com extensão '.sql' a serem
# lidos e executados no datalake
querys = glob( '*.sql' )

# Fazendo conexão com banco de dados que será o destino dos dados consultados
conn = sql3.connect( '../dados.bd' )

In [15]:
# Registrando que só há necessidade de autenticação na primeira consulta do BigQuery
confirmacao = []
for i in range( len( querys ) ):
    if i == 0: confirmacao.append( True  )
    else     : confirmacao.append( False )

In [16]:
# Executando querys e armazenando resultados em arquivos '.csv' na pasta indicada
for i, item in enumerate( querys ):
    (
        # Consultando BigQuery com query dos arquivos '.sql'
        read_sql(
            ler_query( querys[i] ),
            billing_project_id = 'portfolio-367913',
            reauth = confirmacao[i]
        )
        
        # Armazenando dados da consulta em tabelas no banco de dados 'dados.bd'
        .to_sql(
            f'{item[6:-4]}',
            con       = conn,
            if_exists = 'replace',
            index     = False
        )
    )

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=TXuLal2G4CI5kv79JpRFb0Ihcq5BV2&access_type=offline


Downloading: 100%|██████████| 204/204 [00:00<00:00, 621.42rows/s]
